<a href="https://colab.research.google.com/github/reyheneh/10-Essential-Encodings/blob/main/Sequence_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***librarys***

In [ ]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import protpy
from propy import PyPro
from rdkit import Chem
from aaindex import aaindex1
from scipy.fft import fft, fftfreq
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from sklearn.feature_extraction.text import CountVectorizer
from propy.GetProteinFromUniprot import GetProteinSequence

# ***read_peptides***

In [ ]:
def read_data(path):
    all_get_seq = []
    sequences = list(SeqIO.parse(path)) # data_list[0] the index of dataset
    all_get_seq =[]
    for seq_record in sequences:
        c= str(seq_record.seq
            )
        all_get_seq.append(c)
        return all_get_seq


# **Encodings**

**Sparse encoding**

In [ ]:
def encode_sequences(sqs):
    # Define the amino acid alphabet
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    num_sequences = len(sqs)
    max_length = max([len(seq) for seq in sqs])
    matrix = np.zeros((num_sequences, max_length * len(amino_acids)))

    for i, seq in enumerate(sqs):
        vector = np.zeros((len(seq), len(amino_acids)))
        for j, aa in enumerate(seq):
            if aa in amino_acids:
                k = amino_acids.index(aa)
                vector[j,k] = 1
        matrix[i,:len(vector.flatten())] = vector.flatten()
    return matrix


**Amino acid composition**

In [ ]:
def AAC(Data):
    aac = []
    # Loop through each peptide sequence and calculate amino acid composition
    for record in Data:
        pro = protpy.amino_acid_composition(record).values
        aac.extend(pro)

**PseAAC(pseudo amino acid composition)**

In [ ]:
def PseAAC(Data):
    Pse = []
    for seq_data in Data:
        DesObject = PyPro.GetProDes(seq_data)  # construct a GetProDes object
        # calculate 30 pseudo amino acid composition descriptors
        paac = DesObject.GetPAAC(lamda=10, weight=0.05)
        Pse.append(paac)
        return Pse



**Reduced amino acid alphabet**

In [ ]:
aa_to_gbmr4 = {
    'A': 'A',
    'C': 'C',
    'D': 'A',
    'E': 'A',
    'F': 'C',
    'G': 'G',
    'H': 'C',
    'I': 'C',
    'K': 'A',
    'L': 'C',
    'M': 'C',
    'N': 'A',
    'P': 'T',
    'Q': 'A',
    'R': 'A',
    'S': 'A',
    'T': 'A',
    'V': 'C',
    'W': 'C',
    'Y': 'C'}


def Reduced_AA(Data):
    # Define the n-gram length
    n = 3
    # Compute the n-gram counts for each sequence
    n_features = []
    for sequence in Data:
        counts = []
        for i in range(len(sequence) - n + 1):
            n_gram = ''.join([aa_to_gbmr4[sequence[j]] for j in range(i, i + n)])
            counts.append(n_gram)
        n_features.append(counts)


    # Convert n-grams into matrix representation
    vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','), lowercase=False)
    features_mat = vectorizer.fit_transform([','.join(counts) for counts in n_features])
    return features_mat


**Physicochemical properties**

In [ ]:
# Define the physicochemical properties
pI_index = aaindex1["ZIMJ680104"].values  # Isoelectric point
helix_index = aaindex1["BURA740101"].values  # Normalized frequency of alpha-helix
sheet_index = aaindex1["CHOP780202"].values  # Normalized frequency of beta-sheet
turn_index = aaindex1["MONM990101"].values  # sec_struct: Turn propensity scale for transmembrane helices (Monne et al., 1999)
hydrophobicity_index = aaindex1["ARGP820101"].values

def calculate_average_prop(peptide_sequence, prop_values):
    amino_acid_counts = {}
    amino_acid_prop_sum = {}

    # Count the occurrences of each amino acid and calculate the sum of hydrophobicity(for example) values
    for amino_acid in peptide_sequence:
        amino_acid_counts[amino_acid] = amino_acid_counts.get(amino_acid, 0) + 1
        amino_acid_prop_sum[amino_acid] = amino_acid_prop_sum.get(amino_acid, 0) + prop_values[amino_acid]

    average_props = {}

    # Calculate the average property for each amino acid, multiplying by the count
    for amino_acid in amino_acid_counts:
        average_props[amino_acid] = amino_acid_prop_sum[amino_acid] / amino_acid_counts[amino_acid]

    return average_props



**Autocorrelation**

Moran_autocorrelation

In [ ]:
#using default parameters: lag=30, properties=["CIDH920105", "BHAR880101", "CHAM820101", "CHAM820102"
# , "CHOC760101", "BIGC670101", "CHAM810101", "DAYM780201"], normalize=True

def moron_autocorrelation(Data):
    moron = []
    for moran_en in Data:
        try:
            moran_autocorrelation = protpy.moran_autocorrelation(moran_en)
            moron.extend(moran_autocorrelation.values)
        except ZeroDivisionError:
            continue
    return moron


broto_moreau autocorrelation

In [ ]:

#using default parameters: lag=30, properties=["CIDH920105", "BHAR880101", "CHAM820101", "CHAM820102",
# "CHOC760101", "BIGC670101", "CHAM810101", "DAYM780201"], normalize=True
# MBAuto_CIDH920105_1  MBAuto_CIDH920105_2  MBAuto_CIDH920105_3  MBAuto_CIDH920105_4  MBAuto_CIDH920105_5 ...
# -0.052               -0.104               -0.156               -0.208               0.246 ...

def broto_moreau(Data):
    b_moreau = []
    for bm in Data:
        try:
            broto_m = protpy.moreaubroto_autocorrelation(bm)
            b_moreau.extend(broto_m.values)
        except ZeroDivisionError:
            continue
        return b_moreau


****Fourier Transformation****

In [ ]:
# Fourier Analysis of Encoded Peptides
# Computes Fourier transforms of physicochemical properties of peptide sequences and stores their power spectra.

import numpy as np
import pandas as pd
from scipy.fft import fft, fftfreq


# Function to compute the hydrophobicity index
def compute_hydrophobicity_index(sequence):
    try:
        kyte_doolittle_scale = aaindex1['JURD980101']
        return [kyte_doolittle_scale[aa] for aa in sequence]
    except KeyError as e:
        raise ValueError(f"Invalid amino acid in sequence: {e}")

# Function to compute the charge index
def compute_charge_index(sequence):
    try:
        charge_dict = aaindex1['JOND750102']
        return [charge_dict[aa] for aa in sequence]
    except KeyError as e:
        raise ValueError(f"Invalid amino acid in sequence: {e}")

# Function to compute the polarity index
def compute_polarity_index(sequence):
    try:
        polarity_dict = aaindex1['GRAR740102']
        return [polarity_dict[aa] for aa in sequence]
    except KeyError as e:
        raise ValueError(f"Invalid amino acid in sequence: {e}")

# Function to compute the cysteine index
def compute_cysteine_index(sequence):
    return [1 if aa == 'C' else 0 for aa in sequence]

# Initialize empty lists to store the concatenated power spectra
power_spectra = {
    'hydrophobicity': [],
    'charge': [],
    'polarity': [],
    'cysteine': []
}

# Process each peptide sequence
for peptide_sequence in Data:
    # Compute the indices
    hydrophobicity_index = compute_hydrophobicity_index(peptide_sequence)
    charge_index = compute_charge_index(peptide_sequence)
    polarity_index = compute_polarity_index(peptide_sequence)
    cysteine_index = compute_cysteine_index(peptide_sequence)

    # Compute Fourier transforms and power spectra
    n = len(peptide_sequence)
    hydrophobicity_power = np.abs(fft(hydrophobicity_index)) ** 2
    charge_power = np.abs(fft(charge_index)) ** 2
    polarity_power = np.abs(fft(polarity_index)) ** 2
    cysteine_power = np.abs(fft(cysteine_index)) ** 2

    # Store results
    power_spectra['hydrophobicity'].append(hydrophobicity_power)
    power_spectra['charge'].append(charge_power)
    power_spectra['polarity'].append(polarity_power)
    power_spectra['cysteine'].append(cysteine_power)

# Convert results to data frames
df_hydrophobicity = pd.DataFrame(power_spectra['hydrophobicity'])
df_charge = pd.DataFrame(power_spectra['charge'])
df_polarity = pd.DataFrame(power_spectra['polarity'])
df_cysteine = pd.DataFrame(power_spectra['cysteine'])

# Save to CSV files (optional)
df_hydrophobicity.to_csv('hydrophobicity_power_spectra.csv', index=False)
df_charge.to_csv('charge_power_spectra.csv', index=False)
df_polarity.to_csv('polarity_power_spectra.csv', index=False)
df_cysteine.to_csv('cysteine_power_spectra.csv', index=False)

print("Fourier analysis completed. Results saved as CSV files.")


**Quantitative structure-activity relationship (QSAR)**

In [ ]:
# Define a function to calculate chemical properties of a peptide sequence
def calculate_properties(sequence):

    mol = Chem.MolFromFASTA(sequence) # Convert the peptide sequence to an RDKit molecule to represent chemical compounds
    # Calculate the molecular descriptors for the peptide
    mw = Descriptors.MolWt(mol) # Molecular weight
    logp = Descriptors.MolLogP(mol) # LogP
    hba = Descriptors.NumHAcceptors(mol) # Number of hydrogen bond acceptors
    hbd = Descriptors.NumHDonors(mol) # Number of hydrogen bond donors
    tpsa = Descriptors.TPSA(mol) # Topological polar surface area
    return np.array([ mw, logp , hba ,hbd , tpsa ])



**General structural encodings**

In [ ]:
def GSE(Data):
    # Initialize an empty list to store the result for each peptide sequence
    result = []

    # Loop over the peptides and calculate their SASA
    for data in Data:
        # # Initialize an empty list to store the SASA values for each atom in the peptide sequence
        sasa_list = []
        for line in data:
            # Convert the peptide sequence to a molecule object
            mol = Chem.MolFromSequence(line)

            # Add hydrogens to the molecule and embed it in 3D space
            hmol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(hmol)

            # Calculate the atomic radii and SASA
            radii = rdFreeSASA.classifyAtoms(hmol)
            sasa = rdFreeSASA.CalcSASA(hmol, radii)

            # Append the SASA value to the list
            sasa_list.append(sasa)

        # Append the SASA list as a row to the result list
        result.append(sasa_list)
        return result

